<a href="https://colab.research.google.com/github/SofianeSofiane/Face-Mask-detection-/blob/main/loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D


my_model=Sequential()

my_model.add(Conv2D(64,(3,3),input_shape=(200, 200, 1)))
my_model.add(Activation('relu'))
my_model.add(MaxPooling2D(pool_size=(2,2)))

my_model.add(Conv2D(128,(3,3)))
my_model.add(Activation('relu'))
my_model.add(MaxPooling2D(pool_size=(2,2)))

my_model.add(Flatten())
my_model.add(Dropout(0.5))

my_model.add(Dense(64,activation='relu'))

my_model.add(Dense(2,activation='softmax'))

In [ ]:
from keras.utils.vis_utils import plot_model

plot_model(my_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
import os
import tensorflow
checkpoint_path = "drive/MyDrive/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
import tensorflow as tf
!ls {checkpoint_dir}
latest = tf.train.latest_checkpoint(checkpoint_dir)


checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [ ]:

my_model.load_weights(checkpoint_path)

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
from google.colab.patches import cv2_imshow

text_dict={0:'Mask ON',1:'No Mask'}
rect_color_dict={0:(0,255,0),1:(0,0,255)}

In [ ]:
!pip install face_recognition


     |████████████████████████████████| 100.2MB 74kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=0672d18a82ef7cb274a29465e378508835121dcc8d794e7760dac440b8bb93d3
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [ ]:
import cv2
import numpy as np
import face_recognition

image=cv2.imread('photo.jpg')

img=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

faces = face_recognition.face_locations(image,model='hog')

type=isinstance(faces,tuple)

if type==False:
  for face in faces:
     face_location = np.array(face)

     x = face_location[3]
     y = face_location[0]
     w = face_location[1] - face_location[3]
     h = face_location[2] - face_location[0]

     im = image[y:y + w, x:x + w]
     
     grayscale_image=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
     resized_image=cv2.resize(grayscale_image,(200, 200))
     imags=[]
     imags.append(resized_image)
     imags=np.array(imags)/255.0
     imags=np.reshape(imags,(imags.shape[0],200, 200,1))
     predictions=my_model.predict(imags)
     predictions=np.argmax(predictions)
     label=predictions
     cv2.rectangle(image,(x,y),(x+w,y+h),rect_color_dict[label],2)
     cv2.rectangle(image,(x,y-40),(x+w,y),rect_color_dict[label],-1)
     cv2.putText(image, text_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,0),2)
  cv2_imshow(image)
else:
  print('no faces detected')